In [1]:
from discrete_fuzzy_operators.base.operators.binary_operators.discrete.fuzzy_discrete_binary_operator import DiscreteBinaryOperator
from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_implication_suboperators.r_implication import RImplication
from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_implication_suboperators.sn_implication import SNImplication
from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_implication_operator import DiscreteImplicationOperator

from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_aggregation_suboperators.tnorm import Tnorm
from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_aggregation_suboperators.tconorm import Tconorm

from discrete_fuzzy_operators.builtin_operators.discrete.negations import NegationExamples
from discrete_fuzzy_operators.base.operators.unary_operators.suboperators.fuzzy_negation_operator import DiscreteNegation

from enum import Enum

from tqdm.notebook import tqdm
from typing import List

import numpy

In [2]:
dataset_path = r"C:\Users\Usuario\OneDrive - Universitat de les Illes Balears\UIB\Tesi\Experiments\DiscreteDataset"

In [3]:
n = 4
conjunctions = numpy.load(dataset_path+rf"\N={n}\conjunctions.npy", allow_pickle=True)
implications = numpy.array([(n-numpy.flipud(conjunction)) for conjunction in tqdm(conjunctions, desc="Loading implications")])
    
tnorms_raw = numpy.load(dataset_path+rf"\N={n}\tnorms.npy", allow_pickle=True)
tnorms = [Tnorm(n=n, operator_matrix=tnorm_raw, check_properties_in_load=False) for tnorm_raw in tqdm(tnorms_raw, desc="Loading t-norms")]
    
tconorms_raw = numpy.load(dataset_path+rf"\N={n}\tconorms.npy", allow_pickle=True)
tconorms = [Tconorm(n=n, operator_matrix=tconorm_raw, check_properties_in_load=False) for tconorm_raw in tqdm(tconorms_raw, desc="Loading t-conorms")]
    
negations_raw = numpy.load(dataset_path+rf"\N={n}\negations.npy", allow_pickle=True)
negations = [DiscreteNegation(n=n, operator_vector=negation_raw, check_properties_in_load=False) for negation_raw in tqdm(negations_raw, desc="Loading negations")]

Loading implications:   0%|          | 0/208152 [00:00<?, ?it/s]

Loading t-norms:   0%|          | 0/22 [00:00<?, ?it/s]

Loading t-conorms:   0%|          | 0/22 [00:00<?, ?it/s]

Loading negations:   0%|          | 0/35 [00:00<?, ?it/s]

# Generació de totes les R-implicacions

In [4]:
def array_is_in_list(array: numpy.array, list_of_arrays: numpy.array) -> bool:
    for element in list_of_arrays:
        if (numpy.array_equal(element-array, numpy.zeros_like(array))):
            return True
    return False

r_implications = []

for tnorm in tnorms:
    r_implication = RImplication(n=n, operator_tnorm=tnorm)
    if not array_is_in_list(array=r_implication.operator_matrix, list_of_arrays=r_implications):
        r_implications.append(r_implication.operator_matrix)

print(len(r_implications))

22


In [5]:
r_implications2 = []
for implication_raw in implications:
    implication = DiscreteImplicationOperator(n=n, operator_matrix=implication_raw)
    
    if implication.satisfies_exchange_principle() and implication.satisfies_neutrality_principle() and implication.satisfies_ordering_principle():
        r_implications2.append(implication_raw)
        
print(len(r_implications2))

22


# Generació de totes les $(S,N)$-implicacions

In [6]:
s_implications = []
sn_implications = []

for tconorm in tconorms:
    strong_negation = NegationExamples.get_negation(negation=NegationExamples.CLASSIC, n=n)
    s_implication = SNImplication(n=n, operator_tconorm=tconorm, operator_negation=strong_negation)
    s_implications.append(s_implication.operator_matrix)
    
    for negation in negations:
        sn_implication = SNImplication(n=n, operator_tconorm=tconorm, operator_negation=negation)
        
        if not array_is_in_list(array=sn_implication.operator_matrix, list_of_arrays=sn_implications):
            sn_implications.append(sn_implication.operator_matrix)

print(len(s_implications))
print(len(sn_implications))

22
382


# Classificació d'implicacions

In [7]:
class ImplicationClass(Enum):
    R_IMPLICATION = 0
    SN_IMPLICATION = 1
    R_SN_IMPLICATION = 2
    NONE_CLASS = -10
    
def classify_implication(implication: numpy.array, r_implications: List[numpy.array], sn_implications: List[numpy.array]) -> ImplicationClass:
    # Check if the given implication is an R-implication.
    is_r_implication = False
    if array_is_in_list(array=implication, list_of_arrays=r_implications):
        is_r_implication = True
    
    # Check if the given implication is an (S,N)-implication.
    is_sn_implication = False
    if array_is_in_list(array=implication, list_of_arrays=sn_implications):
        is_sn_implication = True
    
    if is_r_implication and not is_sn_implication:
        return ImplicationClass.R_IMPLICATION
    elif not is_r_implication and is_sn_implication:
        return ImplicationClass.SN_IMPLICATION
    elif is_r_implication and is_sn_implication:
        return ImplicationClass.R_SN_IMPLICATION
    else:
        return ImplicationClass.NONE_CLASS

In [8]:
number_r_sn_implications = 0
candidates = r_implications+sn_implications

for implication in candidates:
    if classify_implication(implication, r_implications, sn_implications) == ImplicationClass.R_SN_IMPLICATION:
        number_r_sn_implications += 1

print(rf"The number of discrete (S,N)-implications and R-implications, for n={n}, is: {number_r_sn_implications}")

The number of discrete (S,N)-implications and R-implications, for n=4, is: 6
